In [ ]:
# music_sources/musicbrainz_client.py
from __future__ import annotations
from dataclasses import dataclass, asdict, field
from typing import List, Optional, Dict, Any
import musicbrainzngs as mb

# ---------- Domain models (source-agnostic) ----------
@dataclass
class Artist:
    mbid: str
    name: str
    sort_name: Optional[str] = None

@dataclass
class Release:
    mbid: str
    title: str
    date: Optional[str] = None
    country: Optional[str] = None
    status: Optional[str] = None

@dataclass
class Recording:
    mbid: str
    title: str
    duration_ms: Optional[int] = None
    artist: Optional[str] = None
    releases: List[Release] = None

    def to_json(self) -> Dict[str, Any]:
        d = asdict(self)
        d["releases"] = [asdict(r) for r in self.releases or []]
        return d

# ---------- Client wrapper ----------
class MusicBrainzClient:


    def __init__(self):
        mb.set_useragent("MusicChatbot", "0.1", "team@yourdomain.com")
        mb.set_rate_limit(1.0)

    @staticmethod
    def _artist_credit_to_str(ac_list) -> str:
        parts = []
        for item in ac_list or []:
            if isinstance(item, str):
                parts.append(item)
            elif isinstance(item, dict):
                parts.append(item.get("name") or item.get("artist", {}).get("name", ""))
        return " ".join(p for p in parts if p).replace("  ", " ").strip()

    def search_recordings(self, query: str | RecordingQuery, limit: int = 10) -> List[Recording]:
        lucene_query = query.to_lucene() if isinstance(query, RecordingQuery) else query
        res = mb.search_recordings(query=lucene_query, limit=limit)

        out: List[Recording] = []
        for r in res.get("recording-list", []):
            rels = [
                Release(
                    mbid=rel.get("id", ""),
                    title=rel.get("title", ""),
                    date=rel.get("date"),
                    country=rel.get("country"),
                    status=rel.get("status"),
                )
                for rel in r.get("release-list", []) or []
            ]
            out.append(Recording(
                mbid=r.get("id", ""),
                title=r.get("title", ""),
                duration_ms=int(r["length"]) if "length" in r else None,
                artist=self._artist_credit_to_str(r.get("artist-credit")),
                releases=rels
            ))
        return out


    def lookup_recording_full(self, mbid: str) -> Recording:
        # Expand with artists + releases + isrcs
        r = mb.get_recording_by_id(mbid, includes=["artists", "releases", "isrcs"])
        rec = r["recording"]
        rels = []
        for rel in rec.get("release-list", []) or []:
            rels.append(Release(
                mbid=rel.get("id", ""),
                title=rel.get("title", ""),
                date=rel.get("date"),
                country=rel.get("country"),
                status=rel.get("status")
            ))
        return Recording(
            mbid=rec.get("id", ""),
            title=rec.get("title", ""),
            duration_ms=int(rec["length"]) if "length" in rec else None,
            artist=self._artist_credit_to_str(rec.get("artist-credit")),
            releases=rels
        )


@dataclass
class RecordingQuery:
    """Builds a Lucene query for MusicBrainz recording search."""
    artist: str | None = None
    title: str | None = None
    release: str | None = None
    isrc: str | None = None
    min_duration_ms: int | None = None
    max_duration_ms: int | None = None
    extra_terms: list[str] = field(default_factory=list)

    def to_lucene(self) -> str:
        parts: list[str] = []
        if self.artist:
            parts.append(f'artist:"{self.artist}"')
        if self.title:
            parts.append(f'recording:"{self.title}"')
        if self.release:
            parts.append(f'release:"{self.release}"')
        if self.isrc:
            parts.append(f'isrc:{self.isrc}')
        if self.min_duration_ms or self.max_duration_ms:
            min_val = self.min_duration_ms or "*"
            max_val = self.max_duration_ms or "*"
            parts.append(f'dur:[{min_val} TO {max_val}]')
        parts.extend(self.extra_terms)
        return " AND ".join(parts) if parts else "*"

In [18]:
import json

# One-time init
mbc = MusicBrainzClient()

# Build query object
query = RecordingQuery(
    artist="The Beatles",
    title="Yesterday",
    min_duration_ms=100000,
    max_duration_ms=200000
)

results = mbc.search_recordings(query, limit=5)
print(json.dumps([r.to_json() for r in results], indent=2, ensure_ascii=False))

AttributeError: type object 'MusicBrainzClient' has no attribute '_initialized'